<a href="https://colab.research.google.com/github/SophiaHe/WomenInDSComp/blob/master/WiDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from numpy import unique
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

# Imputer
#from sklearn.preprocessing import Imputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.utils import plot_model
!pip install dplython
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction) 

     |████████████████████████████████| 716kB 2.8MB/s 
  Created wheel for dplython: filename=dplython-0.0.7-cp36-none-any.whl size=747244 sha256=32eb60d5def31a1f68b8a0286acc4eb9b59c43b158b347db78d0517bf5a387ad
  Stored in directory: /root/.cache/pip/wheels/0b/c5/dc/06ed265021b09111a8e7bd7f7577656a64e1293c10df09e0f5
Successfully built dplython


In [0]:
train = pd.read_csv("training_v2.csv")
test = pd.read_csv("unlabeled.csv")


FileNotFoundError: ignored

In [0]:
# keras
X =  train[['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 
'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']]
y = train['hospital_death']
# retrieve numpy array
X = X.values
y = y.values

# train_dataset[0:2,-1]

# format all fields as string
X = X.astype(str)
# reshape target to be a 2d array
y = y.reshape((len(y), 1))

# prepare input data
def prepare_inputs(X_train, X_test):
	X_train_enc, X_test_enc = list(), list()
	# label encode each column
	for i in range(X_train.shape[1]):
		le = LabelEncoder()
		# le.fit_transform(X_train[:, i])
		# encode
		train_enc = le.fit_transform(X_train[:, i])
		test_enc = le.fit_transform(X_test[:, i])
		# store
		X_train_enc.append(train_enc)
		X_test_enc.append(test_enc)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	#le.fit(y_train)
	y_train_enc = le.fit_transform(y_train)
	y_test_enc = le.fit_transform(y_test)
	return y_train_enc, y_test_enc

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# make output 3d
y_train_enc = y_train_enc.reshape((len(y_train_enc), 1, 1))
y_test_enc = y_test_enc.reshape((len(y_test_enc), 1, 1))

# prepare each input head
in_layers = list()
em_layers = list()
for i in range(len(X_train_enc)):
	# calculate the number of unique inputs
	n_labels = len(unique(X_train_enc[i]))
	# define input layer
	in_layer = Input(shape=(1,))
	# define embedding layer
	em_layer = Embedding(n_labels, 10)(in_layer)
	# store layers
	in_layers.append(in_layer)
	em_layers.append(em_layer)
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# plot graph
plot_model(model, show_shapes=True, to_file='embeddings.png')
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=20, batch_size=16, verbose=2)

In [0]:
X =  test[['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 
'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']]

# retrieve numpy array
X = X.values

# prepare input data
def prepare_inputs2( X_test):
	X_test_enc =  list()
	# label encode each column
	for i in range(X_test.shape[1]):
		le = LabelEncoder()

		test_enc = le.fit_transform(X_test[:, i])
		# store
		X_test_enc.append(test_enc)
	return X_test_enc
# format all fields as string
X = X.astype(str)
# prepare input data
X_test_enc2 = prepare_inputs2(X)


In [0]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))
len(X_test_enc2[0])

X_train_enc_pred =model.predict(X_train_enc)
for i in range(5):
	print("X=%s, Predicted=%s" % (X_train_enc[i], y_train_enc[i]))
 X_test_enc_pred =model.predict(X_test_enc2)

Accuracy: 91.23


In [0]:
X_test_enc_pred = X_test_enc_pred.reshape(-1, 1)

print(X_test_enc_pred[0:3])
from numpy import savetxt
savetxt('NN_CatOnly.csv', X_test_enc_pred, delimiter=',')
# kaggle score: 0.66

[[0.08629763]
 [0.01023304]
 [0.00761101]]


CNN using numeric variables

In [0]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras import backend as K
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

np.random.seed(2017)

In [0]:
# delete columns with >60% missing values
# train = train.drop(['patient_id', 'hospital_id','icu_id', 'encounter_id'], axis=1)
# test = test.drop(['patient_id', 'hospital_id','icu_id', 'encounter_id'], axis=1)

cols_to_delete = train.columns[train.isnull().sum()/len(train) > .60]
cols_to_delete2 = test.columns[test.isnull().sum()/len(test) > .60]
cols_to_delete_final = cols_to_delete.intersection(cols_to_delete2)

print(len(cols_to_delete)) # 66
print(len(cols_to_delete2)) # 68
print(len(cols_to_delete.intersection(cols_to_delete2))) # 66 overlaps

train.drop(cols_to_delete_final, axis = 1, inplace = True)
test.drop(cols_to_delete_final, axis = 1, inplace = True)

66
68
66


In [0]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train2 = train.select_dtypes(include=numerics)
test2 = test.select_dtypes(include=numerics)
print(train2.shape)
print(test2.shape)

X_train = train2.loc[:, train2.columns != 'hospital_death']
y_train = train2['hospital_death']

X_test = test2.loc[:, test2.columns != 'hospital_death']
y_test = test2['hospital_death']


(91713, 108)
(39308, 108)


In [0]:
# Binary Classification with Sonar Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# split into input (X) and output (Y) variables
X = train2.loc[:, train2.columns != 'hospital_death']
y = train2['hospital_death']

# baseline model
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(107, input_dim=107, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
X_train = X_train.as_matrix()
X_train = np.reshape(X_train[:,1:108],(53,114,1609,1))

y_train = y_train.as_matrix()
print(y_train.shape)
print(X_train.shape)



(91713,)
(53, 114, 1609, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
# X_test = X_test.as_matrix()
X_test = np.reshape(X_test[:,1:108],(248,53,317,1))
print(X_test.shape)

(248, 53, 317, 1)


In [0]:






Y_train = np_utils.to_categorical(y_train, 2)


print('train labels:\t',Y_train.shape)
print(Y_train[0:10, :])

trainX_normal = (X_train/255.)
testX_normal = (X_test/255.)

trainX_normal = trainX_normal.astype('float32')
testX_normal = testX_normal.astype('float32')

print('train)\tmin: ', np.min(trainX_normal), '\tmax: ', np.max(trainX_normal))
print('test)\tmin: ', np.min(testX_normal), '\tmax: ', np.max(testX_normal))
print(trainX_normal.shape)